In [ ]:
%pip install langchain openai chromadb langchain-community -q

In [8]:
import chromadb

In [9]:
client = chromadb.HttpClient(
    host = "3.35.104.197",
    port = 10090,
)

In [7]:
client.heartbeat()

ConnectTimeout: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다

In [40]:
client.list_collections()

['ingredient',
 'brand',
 'data',
 'posts',
 'posts_copy',
 'posts_copy_2',
 'hyqe',
 'cosmetic']

In [17]:

# 각 컬렉션의 UUID 가져오기
for name in collections:
    collection = client.get_collection(name)
    print(f"Collection Name: {name}, Access ID: {collection.id}")

In [6]:
collection = client.get_collection("posts")

In [ ]:
from pprint import pprint
# # 메타데이터 기반 검색
# result1 = collection.get(
#     where={"doc_id": '8a49cb24-67fd-483c-aa2f-cbb29bf86545'}
# )
doc = collection.get(ids=["16bda3f4-3ab5-42f8-ac37-9822213a7204"])  
pprint(doc)

In [ ]:
collection.get()['ids']['16bda3f4-3ab5-42f8-ac37-9822213a7204']

## OpenAI 조회

## retriever

In [4]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

llm = ChatOpenAI(model_name="gpt-4o", temperature=0, model_kwargs={"stream": True})

In [8]:
from langchain_openai.embeddings import OpenAIEmbeddings
import chromadb
from langchain_chroma import Chroma
from transformers import AutoTokenizer

In [56]:
retriever = vector_store.as_retriever(search_kwargs={"k":3})

In [ ]:
from typing import Dict, Any, List
import json
import chromadb
from langchain.tools import tool
from langchain.agents import  initialize_agent
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel, ValidationError
from typing import Optional

# ChromaDB 클라이언트 설정
chroma_client = chromadb.HttpClient(
    host = "3.35.104.197",
    port = 10090,
)
# LangChain Agent를 위한 OpenAI 모델 설정
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

def get_fresh_llm():
    """매번 새로운 LLM 인스턴스를 생성하여 초기화"""
    return ChatOpenAI(model_name="gpt-4", temperature=0, cache=False)

embedding_function = OpenAIEmbeddings(
    base_url="https://76mqtyy2wie64y-10050.proxy.runpod.net/v1",
    model="multilingual-e5-large",
    api_key="team3",
    tiktoken_enabled=False,
    embedding_ctx_length=502
)

@tool
def classify_question(question: str) -> str:
    """
    사용자의 질문을 분석하여 적절한 검색 카테고리를 결정합니다.
    """
    prompt = f"""
    사용자의 질문을 분석하여 적절한 검색 카테고리를 결정하세요.
    
    - target_collection: 질문에 가장 적합한 컬렉션 (brand, ingredient, cosmetic, law, posts 중 하나)
    - related_collections: 추가적으로 검색할 컬렉션 (cosmetic, posts 무조건 포함)
    - original_query: 원래 질문
    - 각 컬렉션에서 검색할 query
    - k 값 (기본 4, 특정 요청 시 조정 가능)
    - is_amore (자사 브랜드에 관한 질문일 경우 True, 타사 브랜드 관련일 경우 False)
    
    아모레퍼시픽의 자사브랜드: 
    카테고리: BEAUTY_CARE, 브랜드명(한글):설화수, 브랜드명(영어) sulwhasoo
    카테고리: BEAUTY_CARE, 브랜드명(한글):라네즈, 브랜드명(영어) laneige
    카테고리: BEAUTY_CARE, 브랜드명(한글):이니스프리, 브랜드명(영어) innisfree
    카테고리: BEAUTY_CARE, 브랜드명(한글):에이피뷰티, 브랜드명(영어) ap
    카테고리: BEAUTY_CARE, 브랜드명(한글):헤라, 브랜드명(영어) hera
    카테고리: BEAUTY_CARE, 브랜드명(한글):프리메라, 브랜드명(영어) primera
    카테고리: BEAUTY_CARE, 브랜드명(한글):아이오페, 브랜드명(영어) iope
    카테고리: BEAUTY_CARE, 브랜드명(한글):마몽드, 브랜드명(영어) mamonde
    카테고리: BEAUTY_CARE, 브랜드명(한글):한율, 브랜드명(영어) hanyul
    카테고리: MEDICAL_BEAUTY, 브랜드명(한글):에스트라, 브랜드명(영어) aestura
    카테고리: BEAUTY_CARE, 브랜드명(한글):에스쁘아, 브랜드명(영어) espoir
    카테고리: BEAUTY_CARE, 브랜드명(한글):에뛰드, 브랜드명(영어) etude
    카테고리: HAIR, 브랜드명(한글):려, 브랜드명(영어) ryo
    카테고리: HAIR, 브랜드명(한글):미쟝센, 브랜드명(영어) mise en scene
    카테고리: HAIR, 브랜드명(한글):라보에이치, 브랜드명(영어) laboh
    카테고리: HAIR, 브랜드명(한글):아윤채, 브랜드명(영어) ayunche
    카테고리: HAIR, 브랜드명(한글):아모스프로페셔널, 브랜드명(영어) amos professional
    카테고리: HAIR, 브랜드명(한글):롱테이크, 브랜드명(영어) longtake
    카테고리: BODY, 브랜드명(한글):일리윤, 브랜드명(영어) illiyoon
    카테고리: BODY, 브랜드명(한글):해피바스, 브랜드명(영어) happybath
    카테고리: BODY, 브랜드명(한글):스킨유, 브랜드명(영어) skin u
    카테고리: ORAL_CARE, 브랜드명(한글):메디안, 브랜드명(영어) median
    카테고리: ORAL_CARE, 브랜드명(한글):젠티스트, 브랜드명(영어) gentist
    카테고리: PERFUME, 브랜드명(한글):구딸, 브랜드명(영어) goutal
    카테고리: INNER_BEAUTY, 브랜드명(한글):바이탈뷰티, 브랜드명(영어) vital beauty
    카테고리: TEA_CULTURE, 브랜드명(한글):오설록, 브랜드명(영어) osulloc
    카테고리: BEAUTY_DEVICE, 브랜드명(한글):메이크온, 브랜드명(영어) makeon
    카테고리: BEAUTY_CARE, 브랜드명(한글):오딧세이, 브랜드명(영어) odyssey
    카테고리: BEAUTY_CARE, 브랜드명(한글):비레디, 브랜드명(영어) bready
    카테고리: BEAUTY_CARE, 브랜드명(한글):홀리추얼, 브랜드명(영어) holitual
    
    
    질문: {question}
    결과를 JSON 형태로 출력하세요.
    """
    llm = get_fresh_llm() 
    response = llm.invoke(prompt)  # OpenAI API 사용 시 .invoke() 대신 .predict() 사용
    
    response_text = response.content if hasattr(response, "content") else response["content"]
    
    try:
        classification_data = json.loads(response_text.strip())
        
        if not isinstance(classification_data, dict):
            raise ValueError("classify_question 함수: Invalid response format")
        return classification_data
    
    except json.JSONDecodeError:
        return {"error": "JSON 파싱 실패", "raw_response": response}

def retrieve_documents(collection_name: str, query: str, k: int, is_amore: bool = None) -> List[Dict[str, Any]]:
    """
    ChromaDB에서 문서를 검색하여 반환합니다.
    """

    collection = chroma_client.get_collection(collection_name)
    filters = None
    
    if collection_name == "cosmetic" and is_amore is not None:
        is_amore={}
        if is_amore == True: filters["scope"] = '자사'
        elif is_amore == False : filters["scope"] = '타사'
        
    query_embedding = embedding_function.embed_query(query)
    
    results = collection.query(query_embeddings=[query_embedding], n_results=k, where=filters)
    return results

class RetrieveAllDocumentsInput(BaseModel):
    classification: str
   

@tool
def retrieve_all_documents(classification: Any) -> str:
    """
    분류된 target collection 및 related collection에서 문서를 검색하여 반환합니다.
    """
    import ast

    result_dict = ast.literal_eval(classification)

    
    classification_dict = result_dict
    
    retrieved_doc = {
        "target_doc":retrieve_documents(
            collection_name=classification_dict.get('target_collection'), 
            query = classification_dict.get('original_query'),
            k=classification_dict.get('k'),
            is_amore=classification_dict.get('is_amore')
        ) ,
        "target_collection": classification_dict.get('target_collection'), 
        "related_doc": {}
    }
    
    retrieved_doc['target_doc']['collection'] = classification_dict.get('target_collection')
    
    related_collections = classification_dict.get('related_collections', [])

    if len(related_collections) > 0 :
        for collection_name in related_collections:
            result =  retrieve_documents(
                collection_name=collection_name,
                query=classification_dict.get('query'),
                k=classification_dict.get('k'),
                is_amore=classification_dict.get('is_amore')
            )
            retrieved_doc['related_doc'][collection_name] =result.get('metadatas')

    return retrieved_doc
      

response_prompt = PromptTemplate(
    template="""당신은 아모레퍼시픽 전문가입니다.

    질문: {question}

    아래 정보를 참조하여 답변을 생성하세요.

    {context}

    답변:""",
    input_variables=["question", "context"],
)

@tool
def generate_response(question: str) -> str:
    """
    검색된 문서를 기반으로 LLM이 응답을 생성하고 참조 문서를 포함하여 반환합니다.
    """
    
    classification = classify_question(question)
    
    # JSON 파싱 실패 시 예외 처리
    if "error" in classification:
        return {"response": "질문 분류 실패", "error": classification["error"]}

    # 🔥 classification이 문자열이면 JSON 변환
    if isinstance(classification, str):  
        try:
            classification = json.loads(classification)
        except json.JSONDecodeError:
            return {"response": "질문 분류 JSON 변환 실패", "error": classification}

    retrieved_doc = retrieve_all_documents(classification)

    
    context = "\n".join([
        "\n".join([getattr(doc, 'page_content', '') for doc in retrieved_doc["target_doc"]]),
        "\n".join([
            getattr(doc, 'page_content', '')[:1000] 
            for col in retrieved_doc["related_doc"] 
            for doc in retrieved_doc["related_doc"][col]
        ]),
    ])
    
    
    prompt = response_prompt.format(question=question, context=context)

    llm = get_fresh_llm()
    response = llm.predict(prompt)

    return {
        "response": response.strip(),
        "reference_docs": retrieved_doc,  # 검색된 문서를 포함하여 반환
    }

# === LangChain Agent 등록 ===
agent = initialize_agent([classify_question, retrieve_all_documents, generate_response], llm, agent="zero-shot-react-description", verbose=True)

def run_agent(question: str) -> Dict[str, Any]:
    """
    LangChain Agent 실행 함수
    """
    return agent.run(question)


In [ ]:
import json
import ast
import chromadb
from typing import Dict, Any, List
from langchain.tools import tool
from langchain.agents import  initialize_agent
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel, ValidationError
from typing import Optional

# ChromaDB 클라이언트 설정
chroma_client = chromadb.HttpClient(
    host = "3.35.104.197",
    port = 10090,
)
# LangChain Agent를 위한 OpenAI 모델 설정
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

def get_fresh_llm():
    """매번 새로운 LLM 인스턴스를 생성하여 초기화"""
    return ChatOpenAI(model_name="gpt-4", temperature=0, cache=False)

embedding_function = OpenAIEmbeddings(
    base_url="https://76mqtyy2wie64y-10050.proxy.runpod.net/v1",
    model="multilingual-e5-large",
    api_key="team3",
    tiktoken_enabled=False,
    embedding_ctx_length=502
)

response_prompt = PromptTemplate(
    template="""당신은 아모레퍼시픽 전문가입니다.

    질문: {question}

    아래 정보를 참조하여 답변을 생성하세요.

    {context}

    답변:""",
    input_variables=["question", "context"],
)


@tool
def classify_question(question: str) -> str:
    """
    사용자의 질문을 분석하여 적절한 검색 카테고리를 결정합니다.
    """
    
    prompt = f"""
    사용자의 질문을 분석하여 적절한 검색 카테고리를 결정하세요.
    - collection_name 종류: (brand, ingredient, cosmetic, law, posts )
    - 아모레퍼시픽의 자사브랜드: 
        카테고리: BEAUTY_CARE, 브랜드명(한글):설화수, 브랜드명(영어) sulwhasoo
        카테고리: BEAUTY_CARE, 브랜드명(한글):라네즈, 브랜드명(영어) laneige
        카테고리: BEAUTY_CARE, 브랜드명(한글):이니스프리, 브랜드명(영어) innisfree
        카테고리: BEAUTY_CARE, 브랜드명(한글):에이피뷰티, 브랜드명(영어) ap
        카테고리: BEAUTY_CARE, 브랜드명(한글):헤라, 브랜드명(영어) hera
        카테고리: BEAUTY_CARE, 브랜드명(한글):프리메라, 브랜드명(영어) primera
        카테고리: BEAUTY_CARE, 브랜드명(한글):아이오페, 브랜드명(영어) iope
        카테고리: BEAUTY_CARE, 브랜드명(한글):마몽드, 브랜드명(영어) mamonde
        카테고리: BEAUTY_CARE, 브랜드명(한글):한율, 브랜드명(영어) hanyul
        카테고리: MEDICAL_BEAUTY, 브랜드명(한글):에스트라, 브랜드명(영어) aestura
        카테고리: BEAUTY_CARE, 브랜드명(한글):에스쁘아, 브랜드명(영어) espoir
        카테고리: BEAUTY_CARE, 브랜드명(한글):에뛰드, 브랜드명(영어) etude
        카테고리: HAIR, 브랜드명(한글):려, 브랜드명(영어) ryo
        카테고리: HAIR, 브랜드명(한글):미쟝센, 브랜드명(영어) mise en scene
        카테고리: HAIR, 브랜드명(한글):라보에이치, 브랜드명(영어) laboh
        카테고리: HAIR, 브랜드명(한글):아윤채, 브랜드명(영어) ayunche
        카테고리: HAIR, 브랜드명(한글):아모스프로페셔널, 브랜드명(영어) amos professional
        카테고리: HAIR, 브랜드명(한글):롱테이크, 브랜드명(영어) longtake
        카테고리: BODY, 브랜드명(한글):일리윤, 브랜드명(영어) illiyoon
        카테고리: BODY, 브랜드명(한글):해피바스, 브랜드명(영어) happybath
        카테고리: BODY, 브랜드명(한글):스킨유, 브랜드명(영어) skin u
        카테고리: ORAL_CARE, 브랜드명(한글):메디안, 브랜드명(영어) median
        카테고리: ORAL_CARE, 브랜드명(한글):젠티스트, 브랜드명(영어) gentist
        카테고리: PERFUME, 브랜드명(한글):구딸, 브랜드명(영어) goutal
        카테고리: INNER_BEAUTY, 브랜드명(한글):바이탈뷰티, 브랜드명(영어) vital beauty
        카테고리: TEA_CULTURE, 브랜드명(한글):오설록, 브랜드명(영어) osulloc
        카테고리: BEAUTY_DEVICE, 브랜드명(한글):메이크온, 브랜드명(영어) makeon
        카테고리: BEAUTY_CARE, 브랜드명(한글):오딧세이, 브랜드명(영어) odyssey
        카테고리: BEAUTY_CARE, 브랜드명(한글):비레디, 브랜드명(영어) bready
        카테고리: BEAUTY_CARE, 브랜드명(한글):홀리추얼, 브랜드명(영어) holitual
    - 보편적인 관련성에 대해 설명해주자면 target과 related의 collection_name 아래와 같아.
        target: brand -> related: post랑 cosmetic 
        target: ingredient -> related: cosmetic, post
        target: cosmetic -> related: ingredient, post
        target: post -> related: cosmetic, ingredient
        target: law -> related: cosmetic, post, ingredient
        
    
    1. **target**: 
    - `collection_name`: 질문에 가장 적절한 검색 컬렉션의 이름을 지정하세요.
    - `k`: 기본값 3이며, 특정 요청이 있을 경우 조정하세요.
    - `is_amore`: 질문이 자사 브랜드(Amore)에 관한 것이라면 `True`, 타사 브랜드 관련이라면 `False`를 설정하세요.
    - `query`: 사용자의 질문에서 핵심 키워드를 추출하여 embedding query로 사용할 문자열을 생성하세요.

    2. **related** (배열):  
    - 추가적으로 검색할 필요가 있는 컬렉션 목록을 포함하세요.
    - 각 컬렉션은 `collection_name`, `k`, `is_amore`,`query` 값을 포함해야 합니다.

  
    질문: {question}
    결과를 JSON 형태로 출력하세요.
    """
    
    llm = get_fresh_llm() 
    response = llm.invoke(prompt)  # OpenAI API 사용 시 .invoke() 대신 .predict() 사용
    
    response_text = response.content if hasattr(response, "content") else response["content"]
    
    try:
        classification_data = json.loads(response_text.strip())
        
        if not isinstance(classification_data, dict):
            raise ValueError("classify_question 함수: Invalid response format")
        return classification_data
    
    except json.JSONDecodeError:
        return {"error": "JSON 파싱 실패", "raw_response": response}

def retrieve_documents(collection_name: str, query: str, k: int, is_amore: bool = None) -> List[Dict[str, Any]]:
    """
    ChromaDB에서 문서를 검색하여 반환합니다.
    """
    
    collection = chroma_client.get_collection(collection_name)
    filters = None
    
    if collection_name == "cosmetic" and is_amore is not None:
        is_amore={}
        if is_amore == True: filters["scope"] = '자사'
        elif is_amore == False : filters["scope"] = '타사'
        
    query_embedding = embedding_function.embed_query(query)
    
    results = collection.query(query_embeddings=[query_embedding], n_results=k, where=filters)
    return results

class RetrieveAllDocumentsInput(BaseModel):
    classification: str
   

@tool
def retrieve_all_documents(classification: Any) -> str:
    """
    질문에서 추출한 target collection 과 related collection 정보를 가지고 ChromaDB의 collection들에서 retieve 하여 찾은 문서들 모읍니다.
    모은 문서들을 바탕으로 rerank 하여 LLM 응답을 생성합니다.
    """
    import ast

    result_dict = ast.literal_eval(classification)

    classification_dict = result_dict
    
    target_collection = classification_dict.get('target') 
    related_collections = classification_dict.get('related') 

    retrieved_doc = {
        "target_doc":retrieve_documents(
            collection_name=target_collection.get('collection_name'), 
            query = target_collection.get('query'),
            k=target_collection.get('k'),
            is_amore=target_collection.get('is_amore')
        ) ,
        "related_doc": {}
    }
    
    if len(related_collections) > 0 :
        for collection in related_collections:
            result =  retrieve_documents(
                collection_name=collection.get('collection_name'),
                query=collection.get('query'),
                k=collection.get('k'),
                is_amore=collection.get('is_amore')
            )
            retrieved_doc['related_doc'][collection.get('collection_name')] = result

    return retrieved_doc
      

@tool
def rerank_retrieved_documents(retrieved_documents: str) -> str:
    """
    검색된 문서들을 모아서 distance를 계산하고 가장 유사성이 높은 문서들을 뽑아 rerank 하여 LLM이 응답을 생성합니다.

    """

    retrieved_documents = ast.literal_eval(retrieved_documents)
    
    print(retrieved_documents)
    
    # context = "\n".join([
    #     "\n".join([getattr(doc, 'page_content', '') for doc in retrieved_doc["target_doc"]]),
    #     "\n".join([
    #         getattr(doc, 'page_content', '')[:1000] 
    #         for col in retrieved_doc["related_doc"] 
    #         for doc in retrieved_doc["related_doc"][col]
    #     ]),
    # ])
    
    
    # prompt = response_prompt.format(question=question, context=context)

    # llm = get_fresh_llm()
    # response = llm.predict(prompt)
    return retrieved_documents
    return {
        "response": response.strip(),
        "reference_docs": retrieved_doc,  # 검색된 문서를 포함하여 반환
    }

# === LangChain Agent 등록 ===
agent = initialize_agent([classify_question, retrieve_all_documents, rerank_retrieved_documents], llm, agent="zero-shot-react-description", verbose=True)

def run_agent(question: str) -> Dict[str, Any]:
    """
    LangChain Agent 실행 함수
    """
    return agent.run(question)

In [ ]:
run_agent("설화수에 대해 알려줘")

In [ ]:
# from typing import Dict, Any, List
# import json
# import chromadb
# import pymysql
# from langchain.chat_models import init_chat_model
# from langchain.tools import tool
# import os

# DB_CONFIG = {
#     "host": os.getenv("DB_HOST"),
#     "user": os.getenv("DB_USER"),  
#     "password": os.getenv("DB_PASSWD"),
#     "database": os.getenv("DB_NAME"),
#     "charset": "utf8mb4", 
#     "cursorclass": pymysql.cursors.DictCursor
# }

# # === 최신 LangChain 방식으로 LLM 초기화 ===
# llm = init_chat_model("gpt-4")

# # === ChromaDB 클라이언트 설정 ===
# chroma_client = chromadb.HttpClient(
#     host="3.35.104.197",
#     port=10090,
# )

# # === MariaDB 연결 설정 ===
# def get_mariadb_connection():
#     return pymysql.connect(**DB_CONFIG) 


# # === Step 1: 질문 분류 ===
# @tool
# def get_fresh_llm():
#     """매번 새로운 LLM 인스턴스를 생성하여 초기화"""
#     return ChatOpenAI(model_name="gpt-4", temperature=0, cache=False)

# embedding_function = OpenAIEmbeddings(
#     base_url="https://76mqtyy2wie64y-10050.proxy.runpod.net/v1",
#     model="multilingual-e5-large",
#     api_key="team3",
#     tiktoken_enabled=False,
#     embedding_ctx_length=502
# )

# def classify_question(question: str) -> str:
#     """
#     사용자의 질문을 분석하여 적절한 검색 카테고리를 결정합니다.
#     """
#     prompt = f"""
#     사용자의 질문을 분석하여 적절한 검색 카테고리를 결정하세요.
    
#     - target_collection: 질문에 가장 적합한 컬렉션 (brand, ingredient, cosmetic, law, posts 중 하나)
#     - related_collections: 추가적으로 검색할 컬렉션 (cosmetic, posts 무조건 포함)
#     - original_query: 원래 질문
#     - 각 컬렉션에서 검색할 query
#     - k 값 (기본 4, 특정 요청 시 조정 가능)
#     - is_amore (자사 브랜드에 관한 질문일 경우 True, 타사 브랜드 관련일 경우 False)
    
#     아모레퍼시픽의 자사브랜드: 
#     카테고리: BEAUTY_CARE, 브랜드명(한글):설화수, 브랜드명(영어) sulwhasoo
#     카테고리: BEAUTY_CARE, 브랜드명(한글):라네즈, 브랜드명(영어) laneige
#     카테고리: BEAUTY_CARE, 브랜드명(한글):이니스프리, 브랜드명(영어) innisfree
#     카테고리: BEAUTY_CARE, 브랜드명(한글):에이피뷰티, 브랜드명(영어) ap
#     카테고리: BEAUTY_CARE, 브랜드명(한글):헤라, 브랜드명(영어) hera
#     카테고리: BEAUTY_CARE, 브랜드명(한글):프리메라, 브랜드명(영어) primera
#     카테고리: BEAUTY_CARE, 브랜드명(한글):아이오페, 브랜드명(영어) iope
#     카테고리: BEAUTY_CARE, 브랜드명(한글):마몽드, 브랜드명(영어) mamonde
#     카테고리: BEAUTY_CARE, 브랜드명(한글):한율, 브랜드명(영어) hanyul
#     카테고리: MEDICAL_BEAUTY, 브랜드명(한글):에스트라, 브랜드명(영어) aestura
#     카테고리: BEAUTY_CARE, 브랜드명(한글):에스쁘아, 브랜드명(영어) espoir
#     카테고리: BEAUTY_CARE, 브랜드명(한글):에뛰드, 브랜드명(영어) etude
#     카테고리: HAIR, 브랜드명(한글):려, 브랜드명(영어) ryo
#     카테고리: HAIR, 브랜드명(한글):미쟝센, 브랜드명(영어) mise en scene
#     카테고리: HAIR, 브랜드명(한글):라보에이치, 브랜드명(영어) laboh
#     카테고리: HAIR, 브랜드명(한글):아윤채, 브랜드명(영어) ayunche
#     카테고리: HAIR, 브랜드명(한글):아모스프로페셔널, 브랜드명(영어) amos professional
#     카테고리: HAIR, 브랜드명(한글):롱테이크, 브랜드명(영어) longtake
#     카테고리: BODY, 브랜드명(한글):일리윤, 브랜드명(영어) illiyoon
#     카테고리: BODY, 브랜드명(한글):해피바스, 브랜드명(영어) happybath
#     카테고리: BODY, 브랜드명(한글):스킨유, 브랜드명(영어) skin u
#     카테고리: ORAL_CARE, 브랜드명(한글):메디안, 브랜드명(영어) median
#     카테고리: ORAL_CARE, 브랜드명(한글):젠티스트, 브랜드명(영어) gentist
#     카테고리: PERFUME, 브랜드명(한글):구딸, 브랜드명(영어) goutal
#     카테고리: INNER_BEAUTY, 브랜드명(한글):바이탈뷰티, 브랜드명(영어) vital beauty
#     카테고리: TEA_CULTURE, 브랜드명(한글):오설록, 브랜드명(영어) osulloc
#     카테고리: BEAUTY_DEVICE, 브랜드명(한글):메이크온, 브랜드명(영어) makeon
#     카테고리: BEAUTY_CARE, 브랜드명(한글):오딧세이, 브랜드명(영어) odyssey
#     카테고리: BEAUTY_CARE, 브랜드명(한글):비레디, 브랜드명(영어) bready
#     카테고리: BEAUTY_CARE, 브랜드명(한글):홀리추얼, 브랜드명(영어) holitual
    
    
#     질문: {question}
#     결과를 JSON 형태로 출력하세요.
#     """
#     llm = get_fresh_llm() 
#     response = llm.invoke(prompt)  # OpenAI API 사용 시 .invoke() 대신 .predict() 사용
    
#     response_text = response.content if hasattr(response, "content") else response["content"]
#     classification_data = json.loads(response_text.strip())
    
#     print(classification_data)
#     return classification_data
  

# # === Step 2: ChromaDB에서 문서 검색 ===
# @tool
# def retrieve_documents(collection_name: str, query: str, k: int) -> List[Dict[str, Any]]:
#     """
#     특정 컬렉션에서 문서를 검색하여 반환합니다.
#     """
#     collection = chroma_client.get_collection(collection_name)
#     results = collection.query(query_texts=[query], n_results=k)
#     return results

# # === Step 3: MariaDB에서 메타데이터 조회 ===
# @tool
# def fetch_metadata_from_mariadb(collection_name: str, document_list: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
#     """
#     검색된 문서 리스트를 기반으로 MariaDB에서 메타데이터를 조회하여 반환합니다.
#     """
#     conn = get_mariadb_connection()
#     cursor = conn.cursor(dictionary=True)

#     metadata_list = []
    
#     # for doc in document_list:
#     #     doc_id = doc.get("id")  # ChromaDB에서 가져온 문서 ID 사용
#     #     cursor.execute("SELECT * FROM metadata_table WHERE doc_id = %s", (doc_id,))
#     #     metadata = cursor.fetchall()
#     #     metadata_list.extend(metadata)

#     conn.close()
#     return metadata_list

# # === Step 4: 최종 응답 생성 ===
# @tool
# def generate_final_response(post_list: List[Dict[str, Any]], cosmetic_list: List[Dict[str, Any]], question: str) -> Dict[str, Any]:
#     """
#     LLM이 모든 정보를 종합하여 최종 응답을 생성합니다.
#     """
#     prompt = f"""
#     사용자의 질문: {question}

#     posts 관련 문서:
#     {json.dumps(post_list, ensure_ascii=False, indent=2)}

#     cosmetic 관련 문서:
#     {json.dumps(cosmetic_list, ensure_ascii=False, indent=2)}

#     위 정보를 바탕으로 사용자의 질문에 대한 최종 답변을 생성하세요.
#     """
#     response = llm.invoke(prompt)
#     return {"llm_response": response.content, "post_list": post_list, "cosmetic_list": cosmetic_list}

# # === Tool Pipeline 실행 함수 ===
# def run_agent(question: str):
#     """
#     전체 Pipeline을 실행하여 최종 결과를 반환하는 함수
#     """
#     # Step 1: 질문 분류
#     classification = classify_question({"question": question})
    
#     print(classification)
    
#     target_collection = classification["target_collection"]
#     related_collections = classification["related_collections"]

#     # Step 2: 문서 검색 (항상 posts와 cosmetic 포함)
#     retrieved_docs = {
#         "target": retrieve_documents.invoke({"collection_name": target_collection, "query": question, "k": 4}),
#         "posts": retrieve_documents.invoke({"collection_name": "posts", "query": question, "k": 4}),
#         "cosmetic": retrieve_documents.invoke({"collection_name": "cosmetic", "query": question, "k": 4}),
#     }

#     # Step 3: MariaDB에서 메타데이터 조회
#     post_metadata = fetch_metadata_from_mariadb.invoke({"collection_name":"post", "document_list": retrieved_docs["posts"]})
#     cosmetic_metadata = fetch_metadata_from_mariadb.invoke({"collection_name":"cosmetic","document_list": retrieved_docs["cosmetic"]})

#     # Step 4: LLM이 최종 응답 생성
#     final_response = generate_final_response.invoke({
#         "post_list": post_metadata,
#         "cosmetic_list": cosmetic_metadata,
#         "question": question
#     })

#     return final_response


# # === 실행 예제 ===
# query = "설화수 브랜드에 대해 알려줘."
# result = run_agent(query)
# pprint(result)


TypeError: BaseTool.__call__() missing 1 required positional argument: 'tool_input'

''


In [ ]:

# def retrieve_documents(collection_name: str, query: str, k: int, is_amore: bool = None) -> List[Dict[str, Any]]:
#     """
#     ChromaDB에서 문서를 검색하여 반환합니다.
#     """

#     collection = chroma_client.get_collection(collection_name)
#     filters = None
    
#     if collection_name == "cosmetic" and is_amore is not None:
#         is_amore={}
#         if is_amore == True: filters["scope"] = '자사'
#         elif is_amore == False : filters["scope"] = '타사'
        
#     query_embedding = embedding_function.embed_query(query)
#     results = collection.query(query_embeddings=[query_embedding], n_results=k, where=filters)
#     return results

    
# def retrieve_all_documents(classification: Dict[str, Any]) -> Dict[str, Any]:
#     """
#     분류된 target collection 및 related collection에서 문서를 검색하여 반환합니다.
#     """
#     retrieved_doc = {
#         "target_doc": retrieve_documents(
#             classification.get('target_collection'), 
#             classification.get('question'),
#             classification.get('k'),
#             classification.get('is_amore')
#         ),
#         "related_doc": {}
#     }

#     related_collections = classification.get('related_collections', [])
    
#     for collection_name in related_collections:
#         retrieved_doc["related_doc"][collection_name] = retrieve_documents(
#             collection_name,
#             classification.get('query'),
#             classification.get('k'),
#             classification.get('is_amore')
#         )
    
#     return retrieved_doc

In [ ]:
run_agent(' 설화수는 어떤 브랜드야')

In [ ]:
# from langchain_openai import ChatOpenAI
# from langchain.agents import AgentType, initialize_agent
# from langchain.tools import tool
# from langchain.prompts import PromptTemplate
# from langchain.embeddings.openai import OpenAIEmbeddings
# import chromadb
# import json

# LLM_MODEL = "gpt-4o"
# COLLECTION_NAME_POST = "posts"
# COLLECTION_NAME_BRAND = "brand"
# COLLECTION_NAME_COSMETIC = "cosmetic"
# COLLECTION_NAME_INGREDIENT = "ingredient"
# COLLECTION_NAME_LAW = "law"

# collections = {
#     "post": client.get_collection(COLLECTION_NAME_POST),
#     "brand": client.get_collection(COLLECTION_NAME_BRAND),
#     "cosmetic": client.get_collection(COLLECTION_NAME_COSMETIC),
#     "ingredient": client.get_collection(COLLECTION_NAME_INGREDIENT),
#     # "law": client.get_collection(COLLECTION_NAME_LAW),
# }

# embedding_function = OpenAIEmbeddings(
#     base_url="https://76mqtyy2wie64y-10050.proxy.runpod.net/v1",
#     model="multilingual-e5-large",
#     api_key="team3",
#     tiktoken_enabled=False,
#     embedding_ctx_length=502
# )

# client = chromadb.HttpClient(host="3.35.104.197", port=10090)

# classification_prompt = PromptTemplate(
#     template="""당신은 아모레퍼시픽 전문가입니다. 다음 질문을 적절한 카테고리로 분류하세요.
    
#     아모레퍼시픽의 자사브랜드: 
#     카테고리: BEAUTY_CARE, 브랜드명(한글):설화수, 브랜드명(영어) sulwhasoo
#     카테고리: BEAUTY_CARE, 브랜드명(한글):라네즈, 브랜드명(영어) laneige
#     카테고리: BEAUTY_CARE, 브랜드명(한글):이니스프리, 브랜드명(영어) innisfree
#     카테고리: BEAUTY_CARE, 브랜드명(한글):에이피뷰티, 브랜드명(영어) ap
#     카테고리: BEAUTY_CARE, 브랜드명(한글):헤라, 브랜드명(영어) hera
#     카테고리: BEAUTY_CARE, 브랜드명(한글):프리메라, 브랜드명(영어) primera
#     카테고리: BEAUTY_CARE, 브랜드명(한글):아이오페, 브랜드명(영어) iope
#     카테고리: BEAUTY_CARE, 브랜드명(한글):마몽드, 브랜드명(영어) mamonde
#     카테고리: BEAUTY_CARE, 브랜드명(한글):한율, 브랜드명(영어) hanyul
#     카테고리: MEDICAL_BEAUTY, 브랜드명(한글):에스트라, 브랜드명(영어) aestura
#     카테고리: BEAUTY_CARE, 브랜드명(한글):에스쁘아, 브랜드명(영어) espoir
#     카테고리: BEAUTY_CARE, 브랜드명(한글):에뛰드, 브랜드명(영어) etude
#     카테고리: HAIR, 브랜드명(한글):려, 브랜드명(영어) ryo
#     카테고리: HAIR, 브랜드명(한글):미쟝센, 브랜드명(영어) mise en scene
#     카테고리: HAIR, 브랜드명(한글):라보에이치, 브랜드명(영어) laboh
#     카테고리: HAIR, 브랜드명(한글):아윤채, 브랜드명(영어) ayunche
#     카테고리: HAIR, 브랜드명(한글):아모스프로페셔널, 브랜드명(영어) amos professional
#     카테고리: HAIR, 브랜드명(한글):롱테이크, 브랜드명(영어) longtake
#     카테고리: BODY, 브랜드명(한글):일리윤, 브랜드명(영어) illiyoon
#     카테고리: BODY, 브랜드명(한글):해피바스, 브랜드명(영어) happybath
#     카테고리: BODY, 브랜드명(한글):스킨유, 브랜드명(영어) skin u
#     카테고리: ORAL_CARE, 브랜드명(한글):메디안, 브랜드명(영어) median
#     카테고리: ORAL_CARE, 브랜드명(한글):젠티스트, 브랜드명(영어) gentist
#     카테고리: PERFUME, 브랜드명(한글):구딸, 브랜드명(영어) goutal
#     카테고리: INNER_BEAUTY, 브랜드명(한글):바이탈뷰티, 브랜드명(영어) vital beauty
#     카테고리: TEA_CULTURE, 브랜드명(한글):오설록, 브랜드명(영어) osulloc
#     카테고리: BEAUTY_DEVICE, 브랜드명(한글):메이크온, 브랜드명(영어) makeon
#     카테고리: BEAUTY_CARE, 브랜드명(한글):오딧세이, 브랜드명(영어) odyssey
#     카테고리: BEAUTY_CARE, 브랜드명(한글):비레디, 브랜드명(영어) bready
#     카테고리: BEAUTY_CARE, 브랜드명(한글):홀리추얼, 브랜드명(영어) holitual

#     질문: "{question}"
    
#     가능한 카테고리:
#     - "ingredient" (성분, 화장품 기능 관련 질문)
#     - "amore_cosmetic" (자사 화장품 검색 질문)
#     - "other_cosmetic" (타사 화장품 검색 질문)
#     - "cosmetic" (브랜드 구별 없이 화장품에 관한 질문)
#     - "company" (회사 전반적인 질문)
#     - "news" (최신 뉴스나 기사 관련 질문)

#     반드시 위의 카테고리 중 하나만 출력하세요.
    
#     카테고리:""",
#     input_variables=["question"],
# )

# def get_fresh_llm():
#     """매번 새로운 LLM 인스턴스를 생성하여 초기화"""
#     return ChatOpenAI(model_name="gpt-4", temperature=0, cache=False)

# @tool
# def classify_question(question: str) -> str:
#     """질문을 적절한 카테고리로 분류합니다."""
#     llm = get_fresh_llm() 
#     response = llm.invoke(classification_prompt.format(question=question))
#     return response.content.strip().replace('"', '')

# @tool
# def multi_collection_retriever(input_data: str) -> str:
#     """
#     사용자의 질문을 기반으로 적절한 ChromaDB 컬렉션에서 검색 후 JSON 형식의 결과를 반환합니다.
#     input_data는 JSON 문자열이며, 'query'와 'category'를 포함해야 합니다.
#     """
#     data = json.loads(input_data) 
#     query = data["query"]
#     category = data["category"]  

#     retrieved_docs = {} 
#     query_embedding = embedding_function.embed_query(query)

#     search_collections = []
#     metadata_filters = {}
    
#     if category == "ingredient":
#         search_collections = ["ingredient", "cosmetic"]
#     elif category == "amore_cosmetic":
#         search_collections = ["ingredient", "cosmetic"]
#         metadata_filters["cosmetic"] = {"scope": {"$eq": "자사"}}
#     elif category == "other_cosmetic":
#         search_collections = ["ingredient", "cosmetic"]
#         metadata_filters["cosmetic"] = {"scope": {"$eq": "타사"}}
#     elif category == "cosmetic":
#         search_collections = ["cosmetic", "ingredient"]
#     elif category == "company":
#         search_collections = ["brand", "posts"]
#     elif category == "news":
#         search_collections = ["posts"]

#     for collection_name in search_collections:
#         # 해당 컬렉션의 필터 가져오기 (없으면 None으로 설정)
#         filter_condition = metadata_filters.get(collection_name, None) 

#         results = collections[collection_name].query(
#             query_embeddings=[query_embedding], 
#             n_results=5,
#             where=filter_condition if filter_condition else None  # ✅ 빈 `{}` 대신 `None`을 전달
#         )

#         retrieved_docs[collection_name] = [
#             {"content": results['documents'][0][i], "metadata": results['metadatas'][0][i]}
#             for i in range(len(results['ids'][0]))
#         ]
        
#     return json.dumps(retrieved_docs, ensure_ascii=False)

# response_prompt = PromptTemplate(
#     template="""당신은 아모레퍼시픽 전문가입니다.

#     질문: {question}

#     아래 정보를 참고하여 답변을 생성하세요.

#     {context}

#     답변:""",
#     input_variables=["question", "context"],
# )

# @tool
# def generate_response(input_data: str) -> str:
#     """
#     검색된 데이터를 바탕으로 LLM이 최종 응답을 생성합니다.
#     input_data는 JSON 형식으로 전달되며, 'question'과 'retrieved_docs'를 포함합니다.
#     """
#     llm = get_fresh_llm() 
#     data = json.loads(input_data)
#     question = data["question"]
#     retrieved_docs = data["retrieved_docs"]

#     context_sections = []
#     for collection_name, docs in retrieved_docs.items():
#         if docs:
#             doc_texts = "\n".join([doc["content"] for doc in docs])
#             context_sections.append(f"### {collection_name.upper()} 관련 정보 ###\n{doc_texts}")

#     context = "\n\n".join(context_sections)[:7000]

#     response = llm.invoke(response_prompt.format(question=question, context=context))
#     return response.content.strip()

# tools = [classify_question, multi_collection_retriever, generate_response]

# agent = initialize_agent(
#     tools=tools,
#     llm=get_fresh_llm(), 
#     agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#     verbose=True
# )

# def agentic_rag_chain(question: str):
#     category = classify_question(question) 
#     retrieved_docs = multi_collection_retriever(json.dumps({"query": question, "category": category}, ensure_ascii=False)) 
#     # response = generate_response(json.dumps({"question": question, "retrieved_docs": json.loads(retrieved_docs)}, ensure_ascii=False))

#     return {
#         "question": question,
#         "category": category,
#         # "response": response,
#         "retrieved_docs": json.loads(retrieved_docs)
#     }


In [542]:
query1 = "아모레퍼시픽 미백제품에 대해서 알려줘"
response1 = agentic_rag_chain(query1)


검색할 카테고리: amore_cosmetic
검색할 컬렉션: ['ingredient', 'cosmetic']
적용할 메타데이터 필터: {'cosmetic': {'scope': {'$eq': '자사'}}}


In [13]:
from pprint import pprint 

In [ ]:
pprint(response1)

In [83]:

# === 8. 실행 예제 ===
query1 = "미백제품에 대해서 알려줘"
query2 = "설화수 순행클렌징폼과 비슷한 성분을 가진 제품을 알려줘"
query3 = "설화수 순행클렌징폼 성분에 대해 알려줘"

response1 = agentic_rag_chain(query1)
response2 = agentic_rag_chain(query2)
response3 = agentic_rag_chain(query3)


In [84]:
pprint(response1)

{'category': '"cosmetic"', 'question': '미백제품에 대해서 알려줘', 'retrieved_docs': {}}


In [80]:
pprint(response2)

{'category': '"cosmetic"',
 'question': '설화수 순행클렌징폼과 비슷한 성분을 가진 제품을 알려줘',
 'response': "설화수 순행 클렌징폼과 비슷한 성분을 가진 제품으로는 아모레퍼시픽의 '일리윤 세라마이드 아토 수딩 젤'이 있습니다. "
             '이 제품은 설화수 클렌징폼과 같이 세라마이드 성분이 함유되어 있어 피부를 부드럽게 클렌징하면서도 수분을 잃지 않게 '
             '해줍니다. 또한, 민감한 피부에도 사용할 수 있도록 저자극 성분을 사용하였습니다.',
 'retrieved_docs': {}}


In [81]:

pprint(response3)

{'category': '"ingredient"',
 'question': '설화수 순행클렌징폼 성분에 대해 알려줘',
 'response': '설화수 순행클렌징폼은 다양한 천연 성분들로 구성되어 있습니다. 주요 성분으로는 설탕, 글리세린, 스테아릭애씨드, '
             '라우릭애씨드, 팔미틱애씨드, 포타슘하이드록사이드, 마이리스틱애씨드 등이 있습니다. 이 외에도 다양한 식물성 성분들이 '
             '함유되어 있어 피부에 부드럽게 작용하며, 피부를 깨끗하게 클렌징해줍니다.',
 'retrieved_docs': {}}


In [ ]:
# from langchain_openai import ChatOpenAI
# from langchain.agents import AgentType, initialize_agent
# from langchain.tools import StructuredTool
# from langchain.prompts import PromptTemplate
# from langchain.embeddings.openai import OpenAIEmbeddings
# import chromadb
# import json
# from pydantic import BaseModel

# LLM_MODEL = "gpt-4"

# client = chromadb.HttpClient(host="3.35.104.197", port=10090)

# COLLECTIONS = {
#     "posts": client.get_collection("posts"),
#     "brand": client.get_collection("brand"),
#     "cosmetic": client.get_collection("cosmetic"),
#     "ingredient": client.get_collection("ingredient"),
# }

# embedding_function = OpenAIEmbeddings(
#     base_url="https://76mqtyy2wie64y-10050.proxy.runpod.net/v1",
#     model="multilingual-e5-large",
#     api_key="team3",
#     tiktoken_enabled=False,
#     embedding_ctx_length=502
# )

# classification_prompt = PromptTemplate(
#     template="""당신은 아모레퍼시픽 전문가입니다. 다음 질문을 적절한 카테고리로 분류하세요.

#    아모레퍼시픽의 자사브랜드: 
#     카테고리: BEAUTY_CARE, 브랜드명(한글):설화수, 브랜드명(영어) sulwhasoo
#     카테고리: BEAUTY_CARE, 브랜드명(한글):라네즈, 브랜드명(영어) laneige
#     카테고리: BEAUTY_CARE, 브랜드명(한글):이니스프리, 브랜드명(영어) innisfree
#     카테고리: BEAUTY_CARE, 브랜드명(한글):에이피뷰티, 브랜드명(영어) ap
#     카테고리: BEAUTY_CARE, 브랜드명(한글):헤라, 브랜드명(영어) hera
#     카테고리: BEAUTY_CARE, 브랜드명(한글):프리메라, 브랜드명(영어) primera
#     카테고리: BEAUTY_CARE, 브랜드명(한글):아이오페, 브랜드명(영어) iope
#     카테고리: BEAUTY_CARE, 브랜드명(한글):마몽드, 브랜드명(영어) mamonde
#     카테고리: BEAUTY_CARE, 브랜드명(한글):한율, 브랜드명(영어) hanyul
#     카테고리: MEDICAL_BEAUTY, 브랜드명(한글):에스트라, 브랜드명(영어) aestura
#     카테고리: BEAUTY_CARE, 브랜드명(한글):에스쁘아, 브랜드명(영어) espoir
#     카테고리: BEAUTY_CARE, 브랜드명(한글):에뛰드, 브랜드명(영어) etude
#     카테고리: HAIR, 브랜드명(한글):려, 브랜드명(영어) ryo
#     카테고리: HAIR, 브랜드명(한글):미쟝센, 브랜드명(영어) mise en scene
#     카테고리: HAIR, 브랜드명(한글):라보에이치, 브랜드명(영어) laboh
#     카테고리: HAIR, 브랜드명(한글):아윤채, 브랜드명(영어) ayunche
#     카테고리: HAIR, 브랜드명(한글):아모스프로페셔널, 브랜드명(영어) amos professional
#     카테고리: HAIR, 브랜드명(한글):롱테이크, 브랜드명(영어) longtake
#     카테고리: BODY, 브랜드명(한글):일리윤, 브랜드명(영어) illiyoon
#     카테고리: BODY, 브랜드명(한글):해피바스, 브랜드명(영어) happybath
#     카테고리: BODY, 브랜드명(한글):스킨유, 브랜드명(영어) skin u
#     카테고리: ORAL_CARE, 브랜드명(한글):메디안, 브랜드명(영어) median
#     카테고리: ORAL_CARE, 브랜드명(한글):젠티스트, 브랜드명(영어) gentist
#     카테고리: PERFUME, 브랜드명(한글):구딸, 브랜드명(영어) goutal
#     카테고리: INNER_BEAUTY, 브랜드명(한글):바이탈뷰티, 브랜드명(영어) vital beauty
#     카테고리: TEA_CULTURE, 브랜드명(한글):오설록, 브랜드명(영어) osulloc
#     카테고리: BEAUTY_DEVICE, 브랜드명(한글):메이크온, 브랜드명(영어) makeon
#     카테고리: BEAUTY_CARE, 브랜드명(한글):오딧세이, 브랜드명(영어) odyssey
#     카테고리: BEAUTY_CARE, 브랜드명(한글):비레디, 브랜드명(영어) bready
#     카테고리: BEAUTY_CARE, 브랜드명(한글):홀리추얼, 브랜드명(영어) holitual

#     질문: "{question}"
    
#     가능한 카테고리:
#     - "ingredient" (성분, 화장품 기능 관련 질문)
#     - "amore_cosmetic" (자사 화장품 검색 질문)
#     - "other_cosmetic" (타사 화장품 검색 질문)
#     - "cosmetic" (브랜드 구별 없이 화장품에 관한 질문)
#     - "brand" (회사 전반적인 질문)
#     - "posts" (최신 뉴스나 기사 관련 질문)

#     질문: "{question}"
#     카테고리:""",
#     input_variables=["question"],
# )

# def get_fresh_llm():
#     """매번 새로운 LLM 인스턴스를 생성하여 초기화"""
#     return ChatOpenAI(model_name=LLM_MODEL, temperature=0, cache=False)

# class ClassificationInput(BaseModel):
#     question: str

# def classify_question(input_data: ClassificationInput) -> str:
#     """질문을 적절한 카테고리로 분류합니다."""
#     llm = get_fresh_llm() 
#     response = llm.invoke(classification_prompt.format(question=input_data.question))
#     return response.content.strip().replace('"', '')

# class RetrieverInput(BaseModel):
#     query: str
#     category: str

# def multi_collection_retriever(input_data: RetrieverInput) -> str:
#     """
#     사용자의 질문을 기반으로 적절한 ChromaDB 컬렉션에서 검색 후 JSON 형식의 결과를 반환합니다.
#     """
#     retrieved_docs = {} 
#     query_embedding = embedding_function.embed_query(input_data.query)

#     search_collections = []
#     metadata_filters = {}

#     if input_data.category == "ingredient":
#         search_collections = ["ingredient", "cosmetic"]
#     elif input_data.category == "amore_cosmetic":
#         search_collections = ["ingredient", "cosmetic"]
#         metadata_filters["cosmetic"] = {"scope": {"$eq": "자사"}}
#     elif input_data.category == "other_cosmetic":
#         search_collections = ["ingredient", "cosmetic"]
#         metadata_filters["cosmetic"] = {"scope": {"$eq": "타사"}}
#     elif input_data.category == "cosmetic":
#         search_collections = ["cosmetic", "ingredient"]
#     elif input_data.category == "brand":
#         search_collections = ["brand", "posts"]
#     elif input_data.category == "posts":
#         search_collections = ["posts"]

#     for collection_name in search_collections:
#         filter_condition = metadata_filters.get(collection_name, None) 
#         results = COLLECTIONS[collection_name].query(
#             query_embeddings=[query_embedding], 
#             n_results=5,
#             where=filter_condition if filter_condition else None
#         )

#         retrieved_docs[collection_name] = [
#             {"content": results['documents'][0][i], "metadata": results['metadatas'][0][i]}
#             for i in range(len(results['ids'][0]))
#         ]
        
#     return json.dumps(retrieved_docs, ensure_ascii=False)

# response_prompt = PromptTemplate(
#     template="""당신은 아모레퍼시픽 전문가입니다.

#     질문: {question}

#     아래 정보를 참고하여 답변을 생성하세요.

#     {context}

#     답변:""",
#     input_variables=["question", "context"],
# )

# class ResponseInput(BaseModel):
#     question: str
#     retrieved_docs: dict
    
# def generate_response(input_data: ResponseInput) -> str:
#     """검색된 데이터를 바탕으로 LLM이 최종 응답을 생성합니다."""
#     if not input_data.retrieved_docs or all(len(docs) == 0 for docs in input_data.retrieved_docs.values()):
#         return f"관련 정보를 찾을 수 없습니다. '{input_data.question}'에 대한 데이터가 없습니다."

#     llm = get_fresh_llm()
#     context_sections = []
    
#     for collection_name, docs in input_data.retrieved_docs.items():
#         if docs:
#             doc_texts = "\n".join([doc["content"] for doc in docs])
#             context_sections.append(f"### {collection_name.upper()} 관련 정보 ###\n{doc_texts}")

#     context = "\n\n".join(context_sections)[:7000]
#     response = llm.invoke(response_prompt.format(question=input_data.question, context=context))
    
#     return response.content.strip()

# tools = [
#     StructuredTool.from_function(classify_question),
#     StructuredTool.from_function(multi_collection_retriever),
#     StructuredTool.from_function(generate_response)
# ]

# agent = initialize_agent(
#     tools=tools,
#     llm=get_fresh_llm(),
#     agent=AgentType.OPENAI_FUNCTIONS,
#     verbose=True
# )

# def agentic_rag_chain(question: str):
#     """Function Calling을 활용한 최적의 RAG 파이프라인"""
#     category = classify_question(ClassificationInput(question=question))
#     retrieved_docs = multi_collection_retriever(
#         RetrieverInput(query=question, category=category)
#     )
#     response = generate_response(
#         ResponseInput(question=question, retrieved_docs=json.loads(retrieved_docs))
#     )

#     return {
#         "question": question,
#         "category": category,
#         "retrieved_docs": json.loads(retrieved_docs),
#         "response": response
#     }


In [ ]:
# result = agentic_rag_chain('설화수 미백 제품 소개해줘')


# 밑에서 부터 보시면 됩니다 😊

In [15]:
import os
import json
import aiomysql
import asyncio
import chromadb
import numpy as np
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.tools import tool
from langchain_core.messages import SystemMessage
from collections import defaultdict

from pprint import pprint

# Chroma 클라이언트 및 컬렉션 초기화
client = chromadb.HttpClient(host="3.35.104.197", port=10090)

ingredient_store = client.get_collection('ingredient')
cosmetic_store = client.get_collection('cosmetic')
brand_store = client.get_collection('brand')
post_store = client.get_collection('posts')

# 임베딩 함수
embedding_function = OpenAIEmbeddings(
    base_url="https://76mqtyy2wie64y-10050.proxy.runpod.net/v1",
    model="multilingual-e5-large",
    api_key="team3",
    tiktoken_enabled=False,
    embedding_ctx_length=502
)
@tool
def retrieve_ingredient(query: str, k: int = 5):
    """
    화장품 성분 정보를 검색하는 도구. 특정 성분의 효능, 사용 가능 여부, 피부 타입에 따른 추천 정보를 제공합니다.
    "예: '미백 효과가 있는 성분은?', '지성 피부에 좋은 성분 추천해줘', '자외선 차단 성분 알려줘'
    """
    try:
        query_embedding = embedding_function.embed_query(query)
        
        results = ingredient_store.query(query_embeddings=query_embedding, n_results=k)
        return results if results['documents'] else "관련 정보를 찾을 수 없습니다."
    except Exception as e:
        return f"Error retrieving ingredients: {str(e)}"

@tool
def retrieve_cosmetic(query: str, k: int = 3):
    """
    화장품 제품 정보를 검색하는 도구. 특정 기능, 브랜드, 피부 타입에 따라 적절한 화장품을 추천합니다. 
    예: '미백 효과가 있는 화장품 추천해줘', '여드름 피부에 좋은 제품 3개 알려줘', '자사 브랜드 제품 소개해줘'
    """
    try:
        query_embedding = embedding_function.embed_query(query)
        
        results = cosmetic_store.query(query_embeddings=query_embedding, n_results=k)
        return results if results['documents'] else "관련 정보를 찾을 수 없습니다."
    except Exception as e:
        return f"Error retrieving products: {str(e)}"

@tool
def retrieve_brand(query: str, k: int = 5):
    """
    화장품 브랜드 정보를 검색하는 도구. 브랜드의 특징, 주요 제품, 철학(예: 비건, 유기농 등)을 제공합니다.
    예: '이니스프리 브랜드 스토리 알려줘', '비건 화장품 브랜드 추천해줘', '헤어 관련 자사 브랜드 알려줘'
    """
    try:
        query_embedding = embedding_function.embed_query(query)
        
        results = brand_store.query(query_embeddings=query_embedding, n_results=k)
        return results if results['documents'] else "관련 정보를 찾을 수 없습니다."
    except Exception as e:
        return f"Error retrieving brands: {str(e)}"

@tool
def retrieve_posts(query: str, k: int = 5):
    """
    최근 화장품 관련 뉴스, 트렌드, 연구 논문을 검색하는 도구. 뷰티 업계 최신 소식을 제공합니다. 
    예: '최근 트렌드인 화장품 성분이 뭐야?', '화장품 관련 연구 논문 2개 찾아줘', '친환경 화장품 트렌드 기사 알려줘'
    """
    try:
        query_embedding = embedding_function.embed_query(query)
        
        results = post_store.query(query_embeddings=query_embedding, n_results=k)
        return results if results['documents'] else "관련 정보를 찾을 수 없습니다."
    except Exception as e:
        return f"Error retrieving posts: {str(e)}"

# LLM 초기화
def get_fresh_llm():
    return ChatOpenAI(model_name="gpt-4-turbo", temperature=0, cache=False)

tool_list = [retrieve_ingredient, retrieve_cosmetic, retrieve_brand, retrieve_posts]  # ✅ 모든 도구 추가

llm = get_fresh_llm()
llm_with_tools = llm.bind_tools(tools=tool_list, tool_choice="required")

def get_retrieved_documnets(query: str):
    """LLM과 도구 호출을 처리하는 함수"""

    messages = [
    SystemMessage(content="Analyze the user's query and call **all relevant tools** simultaneously. "
                      "For example, if the user asks for cosmetic, ingredients and research papers, "
                      "you must invoke all the tools at once."),
    HumanMessage(query)
    ]

    ai_msg = llm_with_tools.invoke(messages) 

    tool_outputs = []
    tool_dict = {
        "retrieve_ingredient": retrieve_ingredient,
        "retrieve_cosmetic": retrieve_cosmetic,
        "retrieve_brand": retrieve_brand,
        "retrieve_posts": retrieve_posts
    }

    if hasattr(ai_msg, "tool_calls") and ai_msg.tool_calls:
        for tool_call in ai_msg.tool_calls:
            tool_name = tool_call["name"] 
            tool_args = tool_call["args"]

            if tool_name in tool_dict:
                selected_tool = tool_dict[tool_name]
                tool_output = selected_tool.invoke(input=tool_args)  
                
                tool_outputs.append({"tool_name": tool_name, "output": tool_output})

    print(f"실행된 도구 개수: {len(tool_outputs)}") 
    return tool_outputs



def generate_context(ingredient_data, max_tokens=500):
    """
    성분 데이터를 정렬한 후 \n으로 연결하고, 최대 토큰 수를 초과하지 않도록 조정하는 함수.

    Parameters:
    - ingredient_data (list of dict): 성분 데이터 리스트 (각 항목에 'distance'와 'document' 포함)
    - max_tokens (int): 최대 허용 토큰 수 (기본값 500)

    Returns:
    - final_context (str): 정제된 컨텍스트 문자열
    """
    # 1. 거리값 기준으로 정렬 (작을수록 중요)
    sorted_data = sorted(ingredient_data, key=lambda x: x['distance'])
    
    # 2. 문서를 하나로 합치기 (각 문서를 \n으로 연결)
    all_texts = [item['document'] for item in sorted_data]
    full_context = "\n\n".join(all_texts)

    # 3. 토큰 제한 내에서 문서 자르기
    words = full_context.split()  # 공백 기준으로 단어(토큰) 분리
    if len(words) > max_tokens:
        trimmed_context = " ".join(words[:max_tokens])  # max_tokens 만큼만 유지
    else:
        trimmed_context = full_context  # 제한보다 적으면 그대로 반환

    return trimmed_context

def create_prompt(context, question):
    """
    LLM을 위한 프롬프트를 생성하는 함수.
    
    Parameters:
    - context (str): 정제된 성분 데이터 컨텍스트
    - question (str): 사용자가 물어보는 질문
    
    Returns:
    - prompt (str): LLM을 호출할 최적의 프롬프트
    """
    prompt_template = f"""너는 아모레퍼시픽 회사 전문 비서야. 아래 제공된 성분 정보를 참고하여 질문에 답해줘.
    
    --- [성분 정보] ---
    {context}
    --------------------
    
    사용자의 질문:
    {question}
    
    위 정보를 기반으로 전문가처럼 정확하고 상세하게 답변해줘.
    """
    
    return prompt_template



In [6]:

# # ✅ 실행 예제 (여러 개의 도구 실행 가능)
query = "미백성분 3개와 그와 관련된 화장품 3개 추천좀"
retrieved_collections_docs = get_retrieved_documnets(query)

# # 비동기적으로 이후의 작업이 실행되었으면 좋겠어.
# # 1. 일단 첫번째 함수는 llm 답변을 생성하는 것이야.
# flatten_docs = []
# for collection in retrieved_collections_docs:
#     for index in range(len(collection.get('output').get('distances')[0])):
        
#         doc = {'distance':collection.get('output').get('distances')[0][index],
#          'document':collection.get('output').get('documents')[0][index]
#          }
#         flatten_docs.append(doc)

# context = generate_context(flatten_docs, 9000)
# result1 = create_prompt(context, query)
# llm_init = get_fresh_llm()
# response = llm_init(result1)
# pprint(response.content)

# # # 2. get_retrieved_documnets 참고해서 참고문서 meta데이터를 뽑아서 maraidb조회하는것이야.
# # retrieved_collections_docs을 보고 
# # "retrieve_ingredient" 일 경우 metadata에서 ingredient_id를 가지고 와서 mariadb를 조회하고
# # "retrieve_cosmetic" 일 경우 metadata에서 scope와 cosmetic_id를 가져와서 mariadb를 조회하고
# # "retrieve_posts" 일 경우 metadata에서 scope와 post_id 가져와서 post 테이블에서 db조회하기야


        

실행된 도구 개수: 2


In [ ]:
import aiomysql
import asyncio
import os
from collections import defaultdict

DB_CONFIG = {
# 정보 넣어주세요 꼄상씨
    "host": '15.165.170.3',
    "port": 3306,
    "user": '',  
    "password": 'dkahfptjcl123!123',
    "db": '',
    "autocommit": True,
}

    # """
    # 비동기적으로 MariaDB 쿼리를 실행하고 결과를 반환하는 함수
    # :param query: 실행할 SQL 쿼리 (문자열)
    # :param params: SQL에 바인딩할 파라미터 (튜플 또는 리스트)
    # :return: 조회된 결과 리스트
    # """
    # conn = None
    # result = []
    
    
    # try:
    #     conn = await aiomysql.connect(**DB_CONFIG)
    #     async with conn.cursor(aiomysql.DictCursor) as cur:
    #         await cur.execute(query, params)
    #         result = await cur.fetchall()

    #     conn.close()
    #     return result

    # except aiomysql.Error as e:
    #     print(f"[MariaDB 오류]: {e}")
    #     return None


async def generate_llm_response(query, retrieved_collections_docs):
    """
    LLM을 호출하여 응답을 생성하는 함수 (비동기)
    """
    flatten_docs = []
    for collection in retrieved_collections_docs:
        for index in range(len(collection.get('output').get('distances')[0])):
            doc = {
                'distance': collection.get('output').get('distances')[0][index],
                'document': collection.get('output').get('documents')[0][index]
            }
            flatten_docs.append(doc)

    # 토큰 제한을 고려하여 context 생성
    context = generate_context(flatten_docs, 9000)

    prompt = create_prompt(context, query)

    # LLM 호출
    llm_init = get_fresh_llm()
    response = llm_init.invoke(prompt)
    
    print(" [LLM 응답]:")
    pprint(response.content)
    
    return response.content


async def fetch_mariadb_data(retrieved_collections_docs):
    ingredient_ids = []
    cosmetic_queries = defaultdict(list)  # scope별로 묶기
    post_queries = defaultdict(list)  # scope별로 묶기

    for collection in retrieved_collections_docs:
        tool_name = collection.get('tool_name')
        metadatas = collection.get('output').get('metadatas')[0]

        if tool_name == "retrieve_ingredient":
            for meta in metadatas:
                ingredient_ids.append(meta.get('ingred_id'))
        
        elif tool_name == "retrieve_cosmetic":
            for meta in metadatas:
                scope = meta.get('scope')
                cosmetic_queries[scope].append(meta.get('cosmetic_id'))
        
        elif tool_name == "retrieve_posts":
            for meta in metadatas:
                scope = meta.get('scope')
                post_queries[scope].append(meta.get('post_id'))
                
    ingredient_results = None
    if ingredient_ids:
        ingredient_placeholders = ",".join(["%s"] * len(ingredient_ids))
        ingredient_results = await query_mariadb(
            f"SELECT * FROM ingredient WHERE id IN ({ingredient_placeholders})",
            tuple(ingredient_ids) 
        )

    cosmetic_results = {}
    for scope, cosmetic_ids in cosmetic_queries.items():
        if not cosmetic_ids:
            continue

        table_name = "cosmetic" if scope == '자사' else "cosmetic_external"
        cosmetic_placeholders = ",".join(["%s"] * len(cosmetic_ids))
        cosmetic_results[scope] = await query_mariadb(
            f"SELECT * FROM {table_name} WHERE cosmetic_id IN ({cosmetic_placeholders})",
            tuple(cosmetic_ids) 
        )

    post_results = {}
    for scope, post_ids in post_queries.items():
        if not post_ids:
            continue

        table_name = "post" if scope == 'INTERNAL' else "cosmetic_external"
        post_placeholders = ",".join(["%s"] * len(post_ids)) 
        post_results[scope] = await query_mariadb(
            f"SELECT * FROM {table_name} WHERE post_id IN ({post_placeholders})",
            tuple(post_ids) 
        )

    print(" [MariaDB 조회 결과]:")
    pprint({
        "ingredient": ingredient_results,
        "cosmetic": cosmetic_results,
        "post": post_results
    })

    return {
        "ingredient": ingredient_results,
        "cosmetic": cosmetic_results,
        "post": post_results
    }
    
async def main(query):
    retrieved_collections_docs = retrieved_collections_docs

    # LLM 응답과 DB 조회를 병렬 처리 함
    llm_task = asyncio.create_task(generate_llm_response(query, retrieved_collections_docs))
    db_task = asyncio.create_task(fetch_mariadb_data(retrieved_collections_docs))

    llm_response, db_response = await asyncio.gather(llm_task, db_task)

    return {"llm_response": llm_response, "db_response": db_response}



In [30]:
fetch_mariadb_data(retrieved_collections_docs)

 [MariaDB 조회 결과]:
{'cosmetic': {'타사': <coroutine object query_mariadb at 0x0000019061200E10>},
 'ingredient': <coroutine object query_mariadb at 0x0000019061200D00>,
 'post': {}}


{'ingredient': <coroutine object query_mariadb at 0x0000019061200D00>,
 'cosmetic': {'타사': <coroutine object query_mariadb at 0x0000019061200E10>},
 'post': {}}

In [ ]:
await main('미백제품 알려줘')

실행된 도구 개수: 1
 [LLM 응답]:
('제공된 정보에 따르면, 미백 기능을 가진 제품으로는 다음 두 가지가 있습니다:\n'
 '\n'
 '1. **미남크림 톤업화이트닝 50ml (브랜드: 미프)**\n'
 '   - **가격:** 21,000원\n'
 '   - **용도:** 50ml, 모든 피부용\n'
 '   - **성분 중 미백 관련:** 나이아신아마이드 (피부 톤 개선 및 미백 효과), 상황버섯추출물, 꿀풀추출물 등의 식물 추출물이 '
 '포함되어 있어 피부 진정 및 보호 효과를 제공할 수 있습니다.\n'
 '   - **사용법:** 세안 후 스킨 및 로션으로 피부결을 정돈한 후 적당량을 손바닥에 덜어 얼굴에 잘 펴 발라 줍니다.\n'
 '\n'
 '2. **V&A 미백보습크림 (50ml+20ml앰플) (브랜드: 브이앤에이뷰티)**\n'
 '   - **가격:** 62,000원\n'
 '   - **용도:** 크림 50ml, 앰플 20ml, 모든 피부용\n'
 '   - **성분 중 미백 관련:** 나이아신아마이드, 치자꽃추출물, 병풀추출물, 데이지꽃추출물 등이 포함되어 있어 미백 및 피부 진정 '
 '효과를 제공합니다. 또한, 3-O-에틸아스코빅애씨드와 같은 파생 비타민 C 성분이 피부 톤을 밝게 하고 색소 침착을 개선하는 데 도움을 '
 '줄 수 있습니다.\n'
 '   - **사용법:** 크림은 피부결을 따라 부드럽게 펴 발라주고, 앰플은 아침, 저녁 토너 다음 단계에 사용합니다.\n'
 '\n'
 '이 두 제품 모두 미백 효과를 제공하며, 사용자의 피부 타입과 필요에 따라 선택할 수 있습니다. 미남크림 톤업화이트닝은 일반적인 크림 '
 '형태로 일상적인 사용에 적합하며, V&A 미백보습크림은 추가적인 앰플을 포함하여 더 집중적인 미백 관리가 필요한 경우에 적합할 수 '
 '있습니다.')
[MariaDB 오류]: (1045, "Access denied for user 'Playdata'@'DESKTOP-M66ONG7' (using passw

{'llm_response': '제공된 정보에 따르면, 미백 기능을 가진 제품으로는 다음 두 가지가 있습니다:\n\n1. **미남크림 톤업화이트닝 50ml (브랜드: 미프)**\n   - **가격:** 21,000원\n   - **용도:** 50ml, 모든 피부용\n   - **성분 중 미백 관련:** 나이아신아마이드 (피부 톤 개선 및 미백 효과), 상황버섯추출물, 꿀풀추출물 등의 식물 추출물이 포함되어 있어 피부 진정 및 보호 효과를 제공할 수 있습니다.\n   - **사용법:** 세안 후 스킨 및 로션으로 피부결을 정돈한 후 적당량을 손바닥에 덜어 얼굴에 잘 펴 발라 줍니다.\n\n2. **V&A 미백보습크림 (50ml+20ml앰플) (브랜드: 브이앤에이뷰티)**\n   - **가격:** 62,000원\n   - **용도:** 크림 50ml, 앰플 20ml, 모든 피부용\n   - **성분 중 미백 관련:** 나이아신아마이드, 치자꽃추출물, 병풀추출물, 데이지꽃추출물 등이 포함되어 있어 미백 및 피부 진정 효과를 제공합니다. 또한, 3-O-에틸아스코빅애씨드와 같은 파생 비타민 C 성분이 피부 톤을 밝게 하고 색소 침착을 개선하는 데 도움을 줄 수 있습니다.\n   - **사용법:** 크림은 피부결을 따라 부드럽게 펴 발라주고, 앰플은 아침, 저녁 토너 다음 단계에 사용합니다.\n\n이 두 제품 모두 미백 효과를 제공하며, 사용자의 피부 타입과 필요에 따라 선택할 수 있습니다. 미남크림 톤업화이트닝은 일반적인 크림 형태로 일상적인 사용에 적합하며, V&A 미백보습크림은 추가적인 앰플을 포함하여 더 집중적인 미백 관리가 필요한 경우에 적합할 수 있습니다.',
 'db_response': {'ingredient': None, 'cosmetic': {'타사': None}, 'post': {}}}

In [17]:
embedding_function = OpenAIEmbeddings(
    base_url="https://76mqtyy2wie64y-10050.proxy.runpod.net/v1",
    model="multilingual-e5-large",
    api_key="team3",
    tiktoken_enabled=False,
    embedding_ctx_length=502
)
def search(collection:str, question: str, k:int):
    query_embedding = embedding_function.embed_query(question)
    
    # results = None
    # if collection == 'posts':
    results = post_store.query(query_embeddings=query_embedding, n_results=k)
    # elif collection == 'cosmetic': 
        # results = cosmetic_store.query(query_embeddings=query_embedding, n_results=k)
    # elif collection == 'ingredient':
        # results = ingredient_store.query(query_embeddings=query_embedding, n_results=k)
    print(results)
    return results

In [ ]:
search_

NameError: name 'search_' is not defined

: 

In [16]:
search('post', '미백', 10)

{'ids': [['71d98a50-ccf1-4918-b6ca-a092eafff675', '097d1f07-802b-406d-a958-895b48aacabd', '1b2409eb-f032-42df-ba1a-43f3f7021f0e', 'c34f84da-3376-472d-a0a2-1890894f26e2', '9dab1c35-decb-45b6-a3ff-df99b25588c5', 'da1b96c2-acd3-4c51-b326-3d4a06ff39cc', '8c73dee0-a139-40cd-816a-5ccacb6c0ee3', '7397432d-6a95-4e9f-bb27-6ae5a0c43ba8', '3182034d-3fc6-43ae-8f4f-1b0aeb2e504d', 'd277aef7-8492-44a3-9ee0-45221bfc648f']], 'distances': [[0.3152284622192383, 0.3233914375305176, 0.33172282576560974, 0.33266380429267883, 0.3363100588321686, 0.34166985750198364, 0.3427644968032837, 0.3458864390850067, 0.34680697321891785, 0.34730324149131775]], 'embeddings': None, 'metadatas': [[{'authority_level': 0, 'company_id': 'None', 'doc_id': 'f599b683-b9ff-47eb-98bb-31cbeb02caff', 'document_id': 'None', 'post_ctgry': 'None', 'post_id': 9864, 'post_type': 'NEWS', 'questions': '', 'scope': 'EXTERNAL', 'title': '2017년 10월호 [특집] 모발과 피부색 관장하는 색소세포를 살피다 3'}, {'authority_level': 0, 'company_id': 'None', 'doc_id': 'bdeb6

{'ids': [['71d98a50-ccf1-4918-b6ca-a092eafff675',
   '097d1f07-802b-406d-a958-895b48aacabd',
   '1b2409eb-f032-42df-ba1a-43f3f7021f0e',
   'c34f84da-3376-472d-a0a2-1890894f26e2',
   '9dab1c35-decb-45b6-a3ff-df99b25588c5',
   'da1b96c2-acd3-4c51-b326-3d4a06ff39cc',
   '8c73dee0-a139-40cd-816a-5ccacb6c0ee3',
   '7397432d-6a95-4e9f-bb27-6ae5a0c43ba8',
   '3182034d-3fc6-43ae-8f4f-1b0aeb2e504d',
   'd277aef7-8492-44a3-9ee0-45221bfc648f']],
 'distances': [[0.3152284622192383,
   0.3233914375305176,
   0.33172282576560974,
   0.33266380429267883,
   0.3363100588321686,
   0.34166985750198364,
   0.3427644968032837,
   0.3458864390850067,
   0.34680697321891785,
   0.34730324149131775]],
 'embeddings': None,
 'metadatas': [[{'authority_level': 0,
    'company_id': 'None',
    'doc_id': 'f599b683-b9ff-47eb-98bb-31cbeb02caff',
    'document_id': 'None',
    'post_ctgry': 'None',
    'post_id': 9864,
    'post_type': 'NEWS',
    'questions': '',
    'scope': 'EXTERNAL',
    'title': '2017년 10월호 [